## Семинар 10
# Тема: Предобработка данных: перевод категориальных признаков в числовые

**NOMINAL** - те категории данных, которые не имеют порядка (названия городов) **(ONE-HOT ENCODER/GET DUMMIES)**  <br>
**ORDINAL** - те категории данных, в которых важен порядок (богатый/бедный или Старшая/Средняя) **(LABEL ENCODER)**

In [336]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn import metrics

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder

https://www.kaggle.com/lodetomasi1995/income-classification

### 1. Загрузите датасет `income-classification.csv` и вывеите первые 5 строк.

In [337]:
df = pd.read_csv("/content/income_evaluation.csv")
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


### 2. Выведите информацию о загруженном датафрейме и его размеры.

In [338]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              32561 non-null  int64 
 1    workclass       32561 non-null  object
 2    fnlwgt          32561 non-null  int64 
 3    education       32561 non-null  object
 4    education-num   32561 non-null  int64 
 5    marital-status  32561 non-null  object
 6    occupation      32561 non-null  object
 7    relationship    32561 non-null  object
 8    race            32561 non-null  object
 9    sex             32561 non-null  object
 10   capital-gain    32561 non-null  int64 
 11   capital-loss    32561 non-null  int64 
 12   hours-per-week  32561 non-null  int64 
 13   native-country  32561 non-null  object
 14   income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [339]:
df.shape

(32561, 15)

### 3. Убедитесь, что в данных нет пропущенных значений и выведите названия колонок с типом данных object.

In [340]:
df.isna().sum()

,0
age,0
workclass,0
fnlwgt,0
education,0
education-num,0
marital-status,0
occupation,0
relationship,0
race,0
sex,0


In [341]:
df_object = df.select_dtypes(include=['object'])
df_object.columns

Index([' workclass', ' education', ' marital-status', ' occupation',
       ' relationship', ' race', ' sex', ' native-country', ' income'],
      dtype='object')

### 4. Распределите признаки с типом данных object на номинальные и ординальные. Выведите количество уникальных значений в каждом из них.

In [342]:
ordinals = [' workclass', ' education', ' marital-status', ' sex']
nominals = [' occupation', ' race', ' native-country']

In [343]:
df_object[' marital-status'].value_counts()

,count
marital-status,
Married-civ-spouse,14976
Never-married,10683
Divorced,4443
Separated,1025
Widowed,993
Married-spouse-absent,418
Married-AF-spouse,23


In [344]:
df_object[nominals].nunique()

,0
occupation,15
race,5
native-country,42


### 5. Отделите целевую переменную и замените в ней значения на числовые при помощи LabelEncoder.

In [345]:
y = df[' income']
y.value_counts()

,count
income,
<=50K,24720
>50K,7841


In [346]:
labelencoder = LabelEncoder()
y_encoded = labelencoder.fit_transform(y)
y_encoded

array([0, 0, 0, ..., 0, 0, 1])

In [347]:
pd.Series(y_encoded).value_counts()

,count
0,24720
1,7841


### 6. Создайте копию датафрейма с признаками и замените в нём все категориальные ординальные признаки, а также признак sex на числовые при помощи OrdinalEncoder.

In [348]:
ordinalencoder = OrdinalEncoder()
ordinals_encoded = ordinalencoder.fit_transform(df_object[ordinals])
df_ordinals = pd.DataFrame(ordinals_encoded, columns=ordinals)
df_ordinals.head()

,workclass,education,marital-status,sex
0,7.0,9.0,4.0,1.0
1,6.0,9.0,2.0,1.0
2,4.0,11.0,0.0,1.0
3,4.0,1.0,2.0,1.0
4,4.0,9.0,2.0,0.0


### 7. Создайте копию датафрейма и замените в ней категориальный номинальный признак race на числовой при помощи OneHotEncoder.

In [349]:
df_object[' race'].value_counts()

,count
race,
White,27816
Black,3124
Asian-Pac-Islander,1039
Amer-Indian-Eskimo,311
Other,271


In [350]:
onehotencoder = OneHotEncoder(drop='first', sparse_output=False)
noinals_encoded = onehotencoder.fit_transform(df_object[[' race']])
noinals_encoded

array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]])

In [351]:
onehotencoder.get_feature_names_out()

array([' race_ Asian-Pac-Islander', ' race_ Black', ' race_ Other',
       ' race_ White'], dtype=object)

In [352]:
df_nominals = pd.DataFrame(noinals_encoded, columns = onehotencoder.get_feature_names_out()).astype('int64')
df_nominals.head()

,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White
0,0,0,0,1
1,0,0,0,1
2,0,0,0,1
3,0,1,0,0
4,0,1,0,0


In [353]:
df_ordinals.join(df_nominals)

,workclass,education,marital-status,sex,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White
0,7.0,9.0,4.0,1.0,0,0,0,1
1,6.0,9.0,2.0,1.0,0,0,0,1
2,4.0,11.0,0.0,1.0,0,0,0,1
3,4.0,1.0,2.0,1.0,0,1,0,0
4,4.0,9.0,2.0,0.0,0,1,0,0
...,...,...,...,...,...,...,...,...
32556,4.0,7.0,2.0,0.0,0,0,0,1
32557,4.0,11.0,2.0,1.0,0,0,0,1
32558,4.0,11.0,6.0,0.0,0,0,0,1
32559,4.0,11.0,4.0,1.0,0,0,0,1


### 8. Создайте копию датафрейма и замените в ней категориальный номинальный признак race на числовые при помощи get_dummies.

In [354]:
df_dum = df.copy()
pd.get_dummies(df_dum, columns=[' race'], dtype='int64').head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,sex,capital-gain,capital-loss,hours-per-week,native-country,income,race_ Amer-Indian-Eskimo,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,Male,2174,0,40,United-States,<=50K,0,0,0,0,1
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,Male,0,0,13,United-States,<=50K,0,0,0,0,1
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,Male,0,0,40,United-States,<=50K,0,0,0,0,1
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Male,0,0,40,United-States,<=50K,0,0,1,0,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Female,0,0,40,Cuba,<=50K,0,0,1,0,0


### Задания для самостоятельного выполнения:

1. Загрузите датасет `House_Rent_Dataset.csv` для предсказании арендной платы в зависимости от ряда признаков. Убедитесь, что в датасете нет пропущенных значений. Посмотрите размер датасета.

Поработайте с категориальными признаками:

2. Убедитесь в том, что столбец `Area Locality` является столбцом с практически уникальными значениями, удалите его.
3. Сделайте так, чтобы признаки `Area Type` и `Point of Contact` имели только по два значения. Для этого удалите строки с редко встречающимся третьим значением. Замените оставшиеся значения на `0` и `1`.
4. Определите, сколько значений имеет признак `City`, а сколько `Tenant Preferred`. Сделайте из этих признаков dummy переменные, т.е. замените каждый признак на такое количество признаков, сколько значений он принимает, где в каждом новом признаке на месте его значения стоят единицы, а остальные - нули. После этого по одному из столбцов удалите. Присоедините полученные столбцы к исходному датафрейму, а столбцы `City` и `Tenant Preferred` удалите.
5. Замените признак `Furnishing Status` на числовой при помощи LabelEncoder.
6. В дате `Posted On` оставьте только номер месяца, в котором размещено объявление о сдаче в аренду. Тип данных должен быть - целое число.
7. Превратите признак `Floor` в два признака: `Floor_number`- номер этажа, на котором находится квартира (подвалы обозначьте числом `0`) и `Total_floors`- общее количество этажей в доме, в котором сдаётся квартира. Полученные столбцы переведите в тип int.
8. Сохраните полученный датасет и выведете его.
9. Отделите целевую переменную Rent и разбейте данные на обучающую и тестовую выборки. Обучите модель линейной регрессии на обучающих данных и оцените ее эффективность на тестовом наборе.

1. Загрузите датасет House_Rent_Dataset.csv для предсказании арендной платы в зависимости от ряда признаков. Убедитесь, что в датасете нет пропущенных значений. Посмотрите размер датасета.

In [355]:
data = pd.read_csv("/content/House_Rent_Dataset.csv")
data.head()

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2,10000,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
1,2022-05-13,2,20000,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
2,2022-05-16,2,17000,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
3,2022-07-04,2,10000,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
4,2022-05-09,2,7500,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner


In [356]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Posted On          4746 non-null   object
 1   BHK                4746 non-null   int64 
 2   Rent               4746 non-null   int64 
 3   Size               4746 non-null   int64 
 4   Floor              4746 non-null   object
 5   Area Type          4746 non-null   object
 6   Area Locality      4746 non-null   object
 7   City               4746 non-null   object
 8   Furnishing Status  4746 non-null   object
 9   Tenant Preferred   4746 non-null   object
 10  Bathroom           4746 non-null   int64 
 11  Point of Contact   4746 non-null   object
dtypes: int64(4), object(8)
memory usage: 445.1+ KB


In [357]:
data.isna().sum()

,0
Posted On,0
BHK,0
Rent,0
Size,0
Floor,0
Area Type,0
Area Locality,0
City,0
Furnishing Status,0
Tenant Preferred,0


In [358]:
data.shape

(4746, 12)

2. Убедитесь в том, что столбец Area Locality является столбцом с практически уникальными значениями, удалите его.

In [359]:
data['Area Locality'].value_counts()

,count
Area Locality,
Bandra West,37
Gachibowli,29
Electronic City,24
"Miyapur, NH 9",22
Velachery,22
...,...
Hoysala Nagar,1
Nagarabhavi,1
"Shamanna Garden, Wilson Garden",1


In [360]:
data = data.drop('Area Locality', axis=1)
data.head()

,Posted On,BHK,Rent,Size,Floor,Area Type,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2,10000,1100,Ground out of 2,Super Area,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
1,2022-05-13,2,20000,800,1 out of 3,Super Area,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
2,2022-05-16,2,17000,1000,1 out of 3,Super Area,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
3,2022-07-04,2,10000,800,1 out of 2,Super Area,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
4,2022-05-09,2,7500,850,1 out of 2,Carpet Area,Kolkata,Unfurnished,Bachelors,1,Contact Owner


3. Сделайте так, чтобы признаки Area Type и Point of Contact имели только по два значения. Для этого удалите строки с редко встречающимся третьим значением. Замените оставшиеся значения на 0 и 1.

In [361]:
value_counts_area = data['Area Type'].value_counts()
value_counts_area

,count
Area Type,
Super Area,2446
Carpet Area,2298
Built Area,2


In [362]:
value_counts_contact = data['Point of Contact'].value_counts()
value_counts_contact

,count
Point of Contact,
Contact Owner,3216
Contact Agent,1529
Contact Builder,1


In [363]:
top2_area = value_counts_area.nlargest(2).index.tolist()
top2_area

['Super Area', 'Carpet Area']

In [364]:
top2_contact = value_counts_contact.nlargest(2).index.tolist()
top2_contact

['Contact Owner', 'Contact Agent']

In [365]:
data = data[
    data['Area Type'].isin(top2_area) &
    data['Point of Contact'].isin(top2_contact)
]

In [366]:
data['Area Type'].value_counts()

,count
Area Type,
Super Area,2445
Carpet Area,2298


In [367]:
data['Point of Contact'].value_counts()

,count
Point of Contact,
Contact Owner,3214
Contact Agent,1529


In [368]:
labelencoder = LabelEncoder()

data['Area Type'] = labelencoder.fit_transform(data['Area Type'])
data['Point of Contact'] = labelencoder.fit_transform(data['Point of Contact'])
data.head()

<ipython-input-368-5fe29ca300ce>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Area Type'] = labelencoder.fit_transform(data['Area Type'])
<ipython-input-368-5fe29ca300ce>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Point of Contact'] = labelencoder.fit_transform(data['Point of Contact'])


,Posted On,BHK,Rent,Size,Floor,Area Type,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2,10000,1100,Ground out of 2,1,Kolkata,Unfurnished,Bachelors/Family,2,1
1,2022-05-13,2,20000,800,1 out of 3,1,Kolkata,Semi-Furnished,Bachelors/Family,1,1
2,2022-05-16,2,17000,1000,1 out of 3,1,Kolkata,Semi-Furnished,Bachelors/Family,1,1
3,2022-07-04,2,10000,800,1 out of 2,1,Kolkata,Unfurnished,Bachelors/Family,1,1
4,2022-05-09,2,7500,850,1 out of 2,0,Kolkata,Unfurnished,Bachelors,1,1


4. Определите, сколько значений имеет признак City, а сколько Tenant Preferred. Сделайте из этих признаков dummy переменные, т.е. замените каждый признак на такое количество признаков, сколько значений он принимает, где в каждом новом признаке на месте его значения стоят единицы, а остальные - нули. После этого по одному из столбцов удалите. Присоедините полученные столбцы к исходному датафрейму, а столбцы City и Tenant Preferred удалите.

In [369]:
data['City'].value_counts()

,count
City,
Mumbai,972
Chennai,890
Bangalore,886
Hyderabad,866
Delhi,605
Kolkata,524


In [370]:
data['City'].nunique()

6

In [371]:
data['Tenant Preferred'].value_counts()

,count
Tenant Preferred,
Bachelors/Family,3441
Bachelors,830
Family,472


In [372]:
data['Tenant Preferred'].nunique()

3

In [373]:
city_dummies = pd.get_dummies(data['City'], prefix='City', drop_first=True, dtype=int)
tenant_dummies = pd.get_dummies(data['Tenant Preferred'], prefix='Tenant', drop_first=True, dtype=int)

In [374]:
data = pd.concat([data, city_dummies, tenant_dummies], axis=1)
data.head()

,Posted On,BHK,Rent,Size,Floor,Area Type,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact,City_Chennai,City_Delhi,City_Hyderabad,City_Kolkata,City_Mumbai,Tenant_Bachelors/Family,Tenant_Family
0,2022-05-18,2,10000,1100,Ground out of 2,1,Kolkata,Unfurnished,Bachelors/Family,2,1,0,0,0,1,0,1,0
1,2022-05-13,2,20000,800,1 out of 3,1,Kolkata,Semi-Furnished,Bachelors/Family,1,1,0,0,0,1,0,1,0
2,2022-05-16,2,17000,1000,1 out of 3,1,Kolkata,Semi-Furnished,Bachelors/Family,1,1,0,0,0,1,0,1,0
3,2022-07-04,2,10000,800,1 out of 2,1,Kolkata,Unfurnished,Bachelors/Family,1,1,0,0,0,1,0,1,0
4,2022-05-09,2,7500,850,1 out of 2,0,Kolkata,Unfurnished,Bachelors,1,1,0,0,0,1,0,0,0


In [375]:
data = data.drop(['City', 'Tenant Preferred'], axis=1)
data.head()

,Posted On,BHK,Rent,Size,Floor,Area Type,Furnishing Status,Bathroom,Point of Contact,City_Chennai,City_Delhi,City_Hyderabad,City_Kolkata,City_Mumbai,Tenant_Bachelors/Family,Tenant_Family
0,2022-05-18,2,10000,1100,Ground out of 2,1,Unfurnished,2,1,0,0,0,1,0,1,0
1,2022-05-13,2,20000,800,1 out of 3,1,Semi-Furnished,1,1,0,0,0,1,0,1,0
2,2022-05-16,2,17000,1000,1 out of 3,1,Semi-Furnished,1,1,0,0,0,1,0,1,0
3,2022-07-04,2,10000,800,1 out of 2,1,Unfurnished,1,1,0,0,0,1,0,1,0
4,2022-05-09,2,7500,850,1 out of 2,0,Unfurnished,1,1,0,0,0,1,0,0,0


5. Замените признак Furnishing Status на числовой при помощи LabelEncoder.

In [376]:
labelencoder = LabelEncoder()

data['Furnishing Status'] = labelencoder.fit_transform(data['Area Type'])
data.head()

,Posted On,BHK,Rent,Size,Floor,Area Type,Furnishing Status,Bathroom,Point of Contact,City_Chennai,City_Delhi,City_Hyderabad,City_Kolkata,City_Mumbai,Tenant_Bachelors/Family,Tenant_Family
0,2022-05-18,2,10000,1100,Ground out of 2,1,1,2,1,0,0,0,1,0,1,0
1,2022-05-13,2,20000,800,1 out of 3,1,1,1,1,0,0,0,1,0,1,0
2,2022-05-16,2,17000,1000,1 out of 3,1,1,1,1,0,0,0,1,0,1,0
3,2022-07-04,2,10000,800,1 out of 2,1,1,1,1,0,0,0,1,0,1,0
4,2022-05-09,2,7500,850,1 out of 2,0,0,1,1,0,0,0,1,0,0,0


6. В дате Posted On оставьте только номер месяца, в котором размещено объявление о сдаче в аренду. Тип данных должен быть - целое число.

In [377]:
data['Posted On'] = pd.to_datetime(data['Posted On'])

In [378]:
data['Posted On'] = data['Posted On'].dt.month.astype('int8')
data.head()

,Posted On,BHK,Rent,Size,Floor,Area Type,Furnishing Status,Bathroom,Point of Contact,City_Chennai,City_Delhi,City_Hyderabad,City_Kolkata,City_Mumbai,Tenant_Bachelors/Family,Tenant_Family
0,5,2,10000,1100,Ground out of 2,1,1,2,1,0,0,0,1,0,1,0
1,5,2,20000,800,1 out of 3,1,1,1,1,0,0,0,1,0,1,0
2,5,2,17000,1000,1 out of 3,1,1,1,1,0,0,0,1,0,1,0
3,7,2,10000,800,1 out of 2,1,1,1,1,0,0,0,1,0,1,0
4,5,2,7500,850,1 out of 2,0,0,1,1,0,0,0,1,0,0,0


7. Превратите признак Floor в два признака: Floor_number- номер этажа, на котором находится квартира (подвалы обозначьте числом 0) и Total_floors- общее количество этажей в доме, в котором сдаётся квартира. Полученные столбцы переведите в тип int.

In [383]:
def split_floor(floor_str):
    try:
        if 'ground' in str(floor_str).lower():
            floor_num = 0
            parts = str(floor_str).lower().split(' out of ')
            total_floors = int(parts[1]) if len(parts) > 1 else 1
        else:
            parts = str(floor_str).split(' out of ')
            floor_num = int(parts[0]) if parts[0].isdigit() else 1
            total_floors = int(parts[1]) if len(parts) > 1 and parts[1].isdigit() else 1
        return int(floor_num), int(total_floors)
    except:
        return 0, 1


In [384]:
data['Floor']

,Floor
0,Ground out of 2
1,1 out of 3
2,1 out of 3
3,1 out of 2
4,1 out of 2
...,...
4741,3 out of 5
4742,1 out of 4
4743,3 out of 5
4744,23 out of 34


In [385]:
data[['Floor_number', 'Total_floors']] = data['Floor'].apply(
    lambda x: pd.Series(split_floor(x))
)
data.head()

,Posted On,BHK,Rent,Size,Floor,Area Type,Furnishing Status,Bathroom,Point of Contact,City_Chennai,City_Delhi,City_Hyderabad,City_Kolkata,City_Mumbai,Tenant_Bachelors/Family,Tenant_Family,Floor_number,Total_floors
0,5,2,10000,1100,Ground out of 2,1,1,2,1,0,0,0,1,0,1,0,0,2
1,5,2,20000,800,1 out of 3,1,1,1,1,0,0,0,1,0,1,0,1,3
2,5,2,17000,1000,1 out of 3,1,1,1,1,0,0,0,1,0,1,0,1,3
3,7,2,10000,800,1 out of 2,1,1,1,1,0,0,0,1,0,1,0,1,2
4,5,2,7500,850,1 out of 2,0,0,1,1,0,0,0,1,0,0,0,1,2


In [386]:
data = data.drop('Floor', axis=1)
data.head()

,Posted On,BHK,Rent,Size,Area Type,Furnishing Status,Bathroom,Point of Contact,City_Chennai,City_Delhi,City_Hyderabad,City_Kolkata,City_Mumbai,Tenant_Bachelors/Family,Tenant_Family,Floor_number,Total_floors
0,5,2,10000,1100,1,1,2,1,0,0,0,1,0,1,0,0,2
1,5,2,20000,800,1,1,1,1,0,0,0,1,0,1,0,1,3
2,5,2,17000,1000,1,1,1,1,0,0,0,1,0,1,0,1,3
3,7,2,10000,800,1,1,1,1,0,0,0,1,0,1,0,1,2
4,5,2,7500,850,0,0,1,1,0,0,0,1,0,0,0,1,2


8. Сохраните полученный датасет и выведете его.

In [387]:
data.to_csv('processed_House_Rent_Dataset.csv', index=False)

In [388]:
data.head()

,Posted On,BHK,Rent,Size,Area Type,Furnishing Status,Bathroom,Point of Contact,City_Chennai,City_Delhi,City_Hyderabad,City_Kolkata,City_Mumbai,Tenant_Bachelors/Family,Tenant_Family,Floor_number,Total_floors
0,5,2,10000,1100,1,1,2,1,0,0,0,1,0,1,0,0,2
1,5,2,20000,800,1,1,1,1,0,0,0,1,0,1,0,1,3
2,5,2,17000,1000,1,1,1,1,0,0,0,1,0,1,0,1,3
3,7,2,10000,800,1,1,1,1,0,0,0,1,0,1,0,1,2
4,5,2,7500,850,0,0,1,1,0,0,0,1,0,0,0,1,2


9. Отделите целевую переменную Rent и разбейте данные на обучающую и тестовую выборки. Обучите модель линейной регрессии на обучающих данных и оцените ее эффективность на тестовом наборе.

In [389]:
data = pd.read_csv('processed_House_Rent_Dataset.csv')
data.head()

,Posted On,BHK,Rent,Size,Area Type,Furnishing Status,Bathroom,Point of Contact,City_Chennai,City_Delhi,City_Hyderabad,City_Kolkata,City_Mumbai,Tenant_Bachelors/Family,Tenant_Family,Floor_number,Total_floors
0,5,2,10000,1100,1,1,2,1,0,0,0,1,0,1,0,0,2
1,5,2,20000,800,1,1,1,1,0,0,0,1,0,1,0,1,3
2,5,2,17000,1000,1,1,1,1,0,0,0,1,0,1,0,1,3
3,7,2,10000,800,1,1,1,1,0,0,0,1,0,1,0,1,2
4,5,2,7500,850,0,0,1,1,0,0,0,1,0,0,0,1,2


In [390]:
X = data.drop('Rent', axis=1)
y = data['Rent']

In [391]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,  random_state=42)

In [392]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [393]:
y_pred = model.predict(X_test)

In [396]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [397]:
r2_score(y_test, y_pred)

0.46448673004593477

In [398]:
mean_absolute_error(y_test, y_pred)

22455.25503685214

In [399]:
mean_squared_error(y_test, y_pred)

2242148220.2851224

In [400]:
mean_squared_error(y_test, y_pred)

2242148220.2851224